# Logistic regression on mnist dataset using tensorflow.

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [3]:
x_train = np.array(x_train, np.float32)
x_test =  np.array(x_test, np.float32)

x_train = x_train.reshape(len(x_train),x_train.shape[1] * x_train.shape[2])
print(x_train.shape)

x_test = x_test.reshape(len(x_test),x_test.shape[1] * x_test.shape[2])
print(x_test.shape)

(60000, 784)
(10000, 784)


In [4]:
x_train /= 255
x_test /= 255

In [5]:
# MNIST dataset parameters.

# 0 to 9 digits
num_classes = 10 
# 28*28
num_features = 784 

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50


In [6]:
# STEP 4: SHUFFLING AND BATCHING THE DATA

# Use tf.data API to shuffle and batch data.
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [7]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name = "weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name = "bias")

In [8]:
# STEP 6: DEFINING LOGISTIC REGRESSION AND COST FUNCTION
# Logistic regression (Wx + b).

def logistic_regression(x):
    sm=tf.nn.softmax(tf.matmul(x, W) + b)
    return sm
# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.

    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.
    entropy=tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

    return entropy

In [9]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.

optimizer = tf.optimizers.SGD(learning_rate)

In [10]:
# Optimization process. 

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, b])

    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, b]))

In [11]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    if step % display_step == 0:

        pred = logistic_regression(batch_x)

        loss = cross_entropy(pred, batch_y)

        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 118.024467, accuracy: 0.882812
step: 100, loss: 82.208328, accuracy: 0.902344
step: 150, loss: 92.875076, accuracy: 0.894531
step: 200, loss: 79.828705, accuracy: 0.910156
step: 250, loss: 81.294823, accuracy: 0.917969
step: 300, loss: 88.285782, accuracy: 0.886719
step: 350, loss: 74.616058, accuracy: 0.910156
step: 400, loss: 100.066139, accuracy: 0.855469
step: 450, loss: 209.995285, accuracy: 0.863281
step: 500, loss: 50.300385, accuracy: 0.949219
step: 550, loss: 60.730743, accuracy: 0.953125
step: 600, loss: 118.966293, accuracy: 0.878906
step: 650, loss: 54.776451, accuracy: 0.945312
step: 700, loss: 46.161907, accuracy: 0.953125
step: 750, loss: 168.178131, accuracy: 0.835938
step: 800, loss: 81.314774, accuracy: 0.941406
step: 850, loss: 42.638695, accuracy: 0.957031
step: 900, loss: 54.872108, accuracy: 0.937500
step: 950, loss: 74.742447, accuracy: 0.917969
step: 1000, loss: 82.916252, accuracy: 0.906250


In [12]:
# Test model on validation set.

pred = logistic_regression(x_test)

print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.897200
